# Recap
- Fundamentals of Elasticsearch 
- **elasticsearch_py**: Low level python client for Elasticsearch
- **elasticsearch_dsl_py**: High level python client for Elasticsearch built on top of **elasticsearch_py**

We imported twitter data into Python data structures using **Pandas**:

In [ ]:
# Add code to import csv using Pandas (include instructions to install Python3 and then install pandas)

 and indexed a part of it (2k to 4k tweets, depending upon the user) as follows:

In [ ]:
# Add code to index the dictionary into a running elasticsearch cluster 
# (include instructions to install and run elasticsearch and kibana)
# Also include instructions to install elasticsearch_py and elasticsearch_dsl_py

Since Elasticsearch itself does primarily two operations - **Indexing** and **Searching** - the elasticsearch_dsl_py also has two primary structures to work with - **Document** and **Search**. This python client has a more _pythonic sytanx_, hence offering us the chain syntax. This syntax, while convenient, **needs to be used with caution**. Here are some examples of how the behavior of chains becomes different with different functions: 

In [ ]:
# Add example of s.query()

In [ ]:
# Add example of s.execute()

In [ ]:
# Add example of s.query().query()

While one can index data from any different kinds of sources - json files, csv files, data generated in real time from execution of python code (for instance, for logging purposes) - one particular search scenario is almost always required from a web application point of view, and this scenario is searching through the data that needs to be presented to end users. Therefore, we are going to discuss using Elasticsearch to provide search functionality to the end user of any web application. In the current example, we'll leverage **Django** framework for working with web applications.

In [ ]:
# Instructions to download and install django (python 3)

### Introducing Django

In [ ]:
!python manage.py startapp search


In [ ]:
Edit the settings file and add your app, and the django_elasticsearch_dsl to the INSTALLED_APPS list:
    'django_elasticsearch_dsl',
'search',



Create a file documents.py inside the search app folder and fill it with this information:

In [ ]:
from django_elasticsearch_dsl import DocType, Index
from blog.models import Post

posts = Index('posts')

@posts.doc_type
class PostDocument(DocType):
    class Meta:
        model = Post

        fields = [
            'title',
            'id',
            'slug',
            'image',
            'description',
        ]

In here we create an index called posts and connect the Post model with the engine by subclassing DocType.

We also specify the fields we want to index from the Post model.

In this tutorial we will be searching only by the title, but I also index other fields so we can use them in the result page without accessing the site database. Like the id or slug for creating a link and image for the image path.

The actual Post model looks like this in the example site:

In [ ]:
from django.db import models
from django.utils.text import slugify

class Post(models.Model):

    title = models.CharField(max_length=255, blank=True, null=True)
    description = models.TextField(blank=True, null=True)
    image = models.ImageField(upload_to="post_images")
    body = models.TextField(blank=True, null=True)
    order = models.IntegerField(blank=True, null=True)

    slug = models.SlugField(default='', blank=True)

    def save(self):
        self.slug = slugify(self.title)
        super(Post, self).save()

    def __str__(self):
        return '%s' % self.title


 Run this command to index the blog posts:

In [ ]:
python manage.py search_index --rebuild

Open the search app views.py file and write these lines in it:

In [ ]:
from django.shortcuts import render

from search.documents import PostDocument

def search(request):

    q = request.GET.get('q')

    if q:
        posts = PostDocument.search().query("match", title=q)
    else:
        posts = ''

    return render(request, 'search/search.html', {'posts': posts})


In here we get the search word q and find titles matching that word.

Add url pattern for the search page. In here I add the pattern to the main urls.py file:

In [ ]:
from search import views as search_views # < here

urlpatterns = [
    url(r'^admin/', admin.site.urls),
    url(r'^', include('base.urls')),
    url(r'^search/', search_views.search, name='search'), # < here
] + static(settings.MEDIA_URL, document_root=settings.MEDIA_ROOT)

Create a template in search/templates/search/search.html.

In [ ]:
<form method="get">

  <input id="q" name="q" type="text" placeholder="your search...">

</form>

{% for item in posts %}

  {{ item.id }}
  {{ item.title }}
  {{ item.slug }}
  {{ item.image }}

  <br>

{% endfor %}


In [ ]:
A more well formatted 

In [ ]:
<head>
<link href="https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css" rel="stylesheet">
<link href="/static/searchApp/search.css" rel="stylesheet" type="text/css" media="screen" />
</head>
{% load static %}
{% block content %}
<form class="search" method="get">
  <i class="fa fa-search search-icon"></i>
  <input class="search-input" id="q" {% if request.GET.q %}value="{{ request.GET.q }}"{% endif %} name="q" type="text" placeholder="your search...">
    
</form>

{% for item in posts %}

<a class="post">
  <div class="post__title post__item">
    {{ item.title }}
  </div>
  <div class="post__description post__item">
    {{ item.description }}
  </div>  
</a>
{% endfor %}

{% endblock %}